In [ ]:
# Colab에서 실행
!pip install transformers accelerate bitsandbytes

In [ ]:
from transformers import pipeline

translator = pipeline(
    "translation",
    model="facebook/nllb-200-distilled-600M",
    tokenizer="facebook/nllb-200-distilled-600M",
    src_lang="eng_Latn",
    tgt_lang="kor_Hang"
)

result = translator("hello how are you", max_length=40)
print(result[0]["translation_text"])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

Device set to use cuda:0


안녕하세요


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from transformers import BitsAndBytesConfig

# 1) 8-bit 설정 객체
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# 2) 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained(
    "facebook/nllb-200-distilled-600M",
    src_lang="eng_Latn"
)

# 3) 8-bit 양자화 모델 로드 (device_map="auto" 로 GPU/CPU 최적 분배)
model = AutoModelForSeq2SeqLM.from_pretrained(
    "facebook/nllb-200-distilled-600M",
    quantization_config=bnb_config,
    device_map="auto"
)

# 4) pipeline 생성 (tgt_lang 은 tokenizer 변수로 자동 지정됨)
translator = pipeline(
    "translation",
    model=model,
    tokenizer=tokenizer,
    src_lang="eng_Latn",
    tgt_lang="kor_Hang",
    device_map="auto"
)

# 5) 번역 실행
result = translator("hello how are you", max_length=40)
print(result[0]["translation_text"])


Device set to use cuda:0


안녕하세요


In [ ]:
# 1) 저장할 디렉토리 경로 지정
save_dir = "./quantized_nllb_model"

# 2) 토크나이저 저장
tokenizer.save_pretrained(save_dir)

# 3) 양자화된 모델 저장
model.save_pretrained(save_dir)

print(f"Quantized model saved to {save_dir}")


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Quantized model saved to ./quantized_nllb_model


In [ ]:
# Colab에서 필요한 패키지 설치
# !pip install transformers accelerate bitsandbytes

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from transformers import BitsAndBytesConfig

# 1) 8-bit 양자화 설정
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# 2) 저장된 모델 디렉토리
save_dir = "./quantized_nllb_model"

# 3) 토크나이저 & 모델 로드
tokenizer = AutoTokenizer.from_pretrained(save_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(
    save_dir,
    quantization_config=bnb_config,
    device_map="auto"
)

# 4) 파이프라인 생성
#    a) 영어→한국어
translator_en_to_ko = pipeline(
    "translation",
    model=model,
    tokenizer=tokenizer,
    src_lang="eng_Latn",
    tgt_lang="kor_Hang",
    device_map="auto"
)

#    b) 한국어→영어
translator_ko_to_en = pipeline(
    "translation",
    model=model,
    tokenizer=tokenizer,
    src_lang="kor_Hang",
    tgt_lang="eng_Latn",
    device_map="auto"
)

# 5) 번역 실행 및 출력
res1 = translator_en_to_ko("hello world", max_length=20)
print("EN→KR:", res1[0]["translation_text"])

res2 = translator_ko_to_en("안녕하세요", max_length=20)
print("KR→EN:", res2[0]["translation_text"])


/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:212: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
Device set to use cuda:0
Device set to use cuda:0


EN→KR: 안녕하세요
KR→EN: - Hi, how are you?


In [ ]:
!zip -r quantized_nllb_model.zip quantized_nllb_model

  adding: quantized_nllb_model/ (stored 0%)
  adding: quantized_nllb_model/model.safetensors (deflated 7%)
  adding: quantized_nllb_model/generation_config.json (deflated 34%)
  adding: quantized_nllb_model/tokenizer_config.json (deflated 94%)
  adding: quantized_nllb_model/config.json (deflated 60%)
  adding: quantized_nllb_model/tokenizer.json (deflated 82%)
  adding: quantized_nllb_model/sentencepiece.bpe.model (deflated 51%)
  adding: quantized_nllb_model/special_tokens_map.json (deflated 79%)


In [ ]:
# int4

import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    BitsAndBytesConfig,
    pipeline
)

# 1) 4-bit 양자화 설정 (BNB NF4)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# 2) 토크나이저 로드 (src_lang 지정)
tokenizer = AutoTokenizer.from_pretrained(
    "facebook/nllb-200-distilled-600M",
    src_lang="eng_Latn"
)

# 3) 4-bit 양자화 모델 로드 (device_map="auto" 로 GPU/CPU 자동 분배)
model = AutoModelForSeq2SeqLM.from_pretrained(
    "facebook/nllb-200-distilled-600M",
    quantization_config=bnb_config,
    device_map="auto"
)

# 4) (옵션) 모델·토크나이저 저장
save_dir = "./nllb_int4_quantized"
tokenizer.save_pretrained(save_dir)
model.save_pretrained(save_dir)
print(f"INT4-양자화 모델이 '{save_dir}' 에 저장되었습니다.")

# 5) 번역 파이프라인 생성
translator_en_to_ko = pipeline(
    "translation",
    model=model,
    tokenizer=tokenizer,
    src_lang="eng_Latn",
    tgt_lang="kor_Hang",
    device_map="auto"
)
translator_ko_to_en = pipeline(
    "translation",
    model=model,
    tokenizer=tokenizer,
    src_lang="kor_Hang",
    tgt_lang="eng_Latn",
    device_map="auto"
)

# 6) 테스트 번역
res1 = translator_en_to_ko("hello world", max_length=20)
print("EN→KR:", res1[0]["translation_text"])

res2 = translator_ko_to_en("안녕하세요", max_length=20)
print("KR→EN:", res2[0]["translation_text"])


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
Device set to use cuda:0
Device set to use cuda:0


INT4-양자화 모델이 './nllb_int4_quantized' 에 저장되었습니다.
EN→KR: 안녕하세요
KR→EN: - Hi, how are you?


In [ ]:
!zip -r nllb_int4.zip nllb_int4_quantized/

  adding: nllb_int4_quantized/ (stored 0%)
  adding: nllb_int4_quantized/model.safetensors (deflated 8%)
  adding: nllb_int4_quantized/generation_config.json (deflated 34%)
  adding: nllb_int4_quantized/tokenizer_config.json (deflated 94%)
  adding: nllb_int4_quantized/config.json (deflated 60%)
  adding: nllb_int4_quantized/tokenizer.json (deflated 82%)
  adding: nllb_int4_quantized/sentencepiece.bpe.model (deflated 51%)
  adding: nllb_int4_quantized/special_tokens_map.json (deflated 79%)


In [ ]:
# (1) bitsandbytes, accelerate 설치
!pip install bitsandbytes accelerate optimum[onnxruntime]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 109.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 109.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.3/429.3 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.1 MB/s eta 0:00:00


In [ ]:
# 1) ONNX 내보내기
!optimum-cli export onnx \
  ./model/nllb_onnx/ \
  --model facebook/nllb-200-distilled-600M \
  --task seq2seq-lm


2025-05-22 11:42:37.500978: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747914157.632382    2945 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747914157.675468    2945 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Moving the following attributes in the config to the generation config: {'max_length': 200}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
/usr/local/lib/python3.11/dist-packages/transformers/models/m2m_100/modeling_m2m_100.py:163: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data fl

In [ ]:
# 2) ONNX Runtime 동적 INT8 양자화
!optimum-cli onnxruntime quantize \
  --onnx_model ./model/nllb_onnx/ \
  -o ./model/nllb_int8_quantized/ \
  --avx2


2025-05-22 11:49:44.238845: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747914584.260779    4836 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747914584.267418    4836 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
^C


In [ ]:
!pip install transformers

In [ ]:
!rm -rf /content/onnx_fp32

In [ ]:
!optimum-cli export onnx \
  ./model/onnx_fp32 \
  --model facebook/nllb-200-distilled-600M \
  --task seq2seq-lm


2025-05-22 13:22:56.722746: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747920176.744419   28499 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747920176.751701   28499 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Moving the following attributes in the config to the generation config: {'max_length': 200}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
/usr/local/lib/python3.11/dist-packages/transformers/models/m2m_100/modeling_m2m_100.py:163: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data fl

In [ ]:
!optimum-cli onnxruntime quantize \
  --onnx_model ./model/onnx_fp32 \
  --output ./model/onnx_int8 \
  --avx2


In [ ]:
!zip -r int4.zip /content/nllb_int4_quantized

  adding: content/nllb_int4_quantized/ (stored 0%)
  adding: content/nllb_int4_quantized/model.safetensors (deflated 8%)
  adding: content/nllb_int4_quantized/generation_config.json (deflated 34%)
  adding: content/nllb_int4_quantized/tokenizer_config.json (deflated 94%)
  adding: content/nllb_int4_quantized/config.json (deflated 60%)
  adding: content/nllb_int4_quantized/tokenizer.json (deflated 82%)
  adding: content/nllb_int4_quantized/sentencepiece.bpe.model (deflated 51%)
  adding: content/nllb_int4_quantized/special_tokens_map.json (deflated 79%)
